In [1]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/sentiment-models')

Mounted at /content/drive/


In [2]:
import pandas as pd

df_sub_xlnet = pd.read_csv("./input/sub_xlnet_op.csv")
df_sub_bert = pd.read_csv("./input/sub_bert_op.csv")
df_sub_roberta = pd.read_csv("./input/sub_roberta_op.csv")



In [3]:
df_sub_xlnet["label_xlnet"] = df_sub_xlnet["label"]
df_sub_xlnet["label_bert"] = df_sub_bert["label"]
df_sub_xlnet["label_roberta"] = df_sub_roberta["label"]

df_sub_xlnet.drop(columns="label", axis=1, inplace=True)



In [4]:
xlnet_bert_equals = df_sub_xlnet.loc[df_sub_xlnet['label_bert']==df_sub_xlnet['label_roberta']]



In [5]:
labels = []

for index, row in df_sub_xlnet.iterrows():
    if(row['label_xlnet'] + row['label_bert'] + row['label_roberta'])>= 2:
      labels.append(1)
    else:
      labels.append(0)
    #print(row['label_xlnet'] + row['label_bert'] + row['label_roberta'])
df_sub_xlnet['label'] = labels

df_sub_xlnet.drop(columns="label_xlnet", axis=1, inplace=True)
df_sub_xlnet.drop(columns="label_bert", axis=1, inplace=True)
df_sub_xlnet.drop(columns="label_roberta", axis=1, inplace=True)


In [6]:
# write predictions to a CSV file
df_sub_xlnet.to_csv("ensemble.csv", index=False)

## Loading checkpoint

In [7]:
test_data = pd.read_csv("./input/test_oJQbWVk.csv")
X_test = list(test_data["tweet"])

In [8]:
!pip install transformers
!pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 933.3 kB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.6 MB/s eta 0:00:00


In [9]:
import torch
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [10]:
from transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from transformers import TrainingArguments, Trainer
import numpy as np
from scipy.special import softmax



# Load trained model
model_path = "./model/xlnet-checkpoint-1782"
model = XLNetForSequenceClassification.from_pretrained(model_path, num_labels=2)

model_name = "xlnet-base-cased"
tokenizer = XLNetTokenizer.from_pretrained(model_name, do_lower_case=True)

# Define test trainer
trainer = Trainer(model)

X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create torch dataset
test_dataset = Dataset(X_test_tokenized)

# Calculating probabilities

#probs = []

#pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer)
#for elem in X_test:
#  probs.append(pipe(elem, return_all_scores=True))
#prediction = pipe("The text to predict", return_all_scores=True)


# Make prediction
raw_pred, _, _ = trainer.predict(test_dataset)

probabilities = softmax(raw_pred, axis=1)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

df_xlnet = pd.DataFrame({'xlnet_raw_predictions':raw_pred.tolist(), 'xlnet_probabilities': probabilities.tolist(), "xlnet_label": y_pred})

#df_xlnet
#raw_pred
#sub = pd.DataFrame({'id':test_data['id'], 'label':y_pred})

# write predictions to a CSV file
#sub.to_csv("sub_xlnet_op.csv", index=False)

In [31]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load trained model
model_path = "./model/bert-checkpoint-2673"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

model_name = "bert-large-uncased" #bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# Define test trainer
trainer = Trainer(model)

X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create torch dataset
test_dataset = Dataset(X_test_tokenized)

# Make prediction
raw_pred, _, _ = trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

probabilities = softmax(raw_pred, axis=1)

df_bert = pd.DataFrame({'bert_raw_predictions':raw_pred.tolist(), 'bert_probabilities': probabilities.tolist(), "bert_label": y_pred})

#df_bert


In [28]:
probabilities

array([[0.02477175, 0.97522825],
       [0.02745243, 0.9725476 ],
       [0.03131326, 0.9686867 ],
       ...,
       [0.03425178, 0.96574825],
       [0.08096901, 0.919031  ],
       [0.9988819 , 0.00111813]], dtype=float32)

In [29]:
raw_pred

array([[-2.0532625,  1.619705 ],
       [-1.9895514,  1.577913 ],
       [-1.9293958,  1.5025036],
       ...,
       [-1.8838587,  1.4553062],
       [-1.4371195,  0.9921338],
       [ 3.5484514, -3.246528 ]], dtype=float32)

In [12]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Load trained model
model_path = "./model/roberta-checkpoint-1782"
model = RobertaForSequenceClassification.from_pretrained(model_path, num_labels=2)

model_name = 'roberta-base' #'roberta-large' , 'roberta-large-mnli'

tokenizer = RobertaTokenizer.from_pretrained(model_name)

# Define test trainer
trainer = Trainer(model)

X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create torch dataset
test_dataset = Dataset(X_test_tokenized)

# Make prediction
raw_pred, _, _ = trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

probabilities = softmax(raw_pred, axis=1)

df_roberta = pd.DataFrame({'roberta_raw_predictions':raw_pred.tolist(), 'roberta_probabilities': probabilities.tolist(), "roberta_label": y_pred})

#df_roberta


In [13]:
df_xlnet['bert_raw_predictions'] = df_bert['bert_raw_predictions']
df_xlnet['bert_probabilities'] = df_bert['bert_probabilities']
df_xlnet['bert_label'] = df_bert['bert_label']

df_xlnet['roberta_raw_predictions'] = df_roberta['roberta_raw_predictions']
df_xlnet['roberta_probabilities'] = df_roberta['roberta_probabilities']
df_xlnet['roberta_label'] = df_roberta['roberta_label']

df_xlnet.head()

,xlnet_raw_predictions,xlnet_probabilities,xlnet_label,bert_raw_predictions,bert_probabilities,bert_label,roberta_raw_predictions,roberta_probabilities,roberta_label
0,"[-1.9433740377426147, 2.657467842102051]","[0.009943508543074131, 0.9900564551353455]",1,"[-2.233088970184326, 2.498344659805298]","[0.00873681902885437, 0.9912631511688232]",1,"[-2.05326247215271, 1.6197049617767334]","[0.024771753698587418, 0.9752282500267029]",1
1,"[-0.7860005497932434, 1.0093395709991455]","[0.1424192637205124, 0.8575807809829712]",1,"[-2.0272960662841797, 2.4038662910461426]","[0.011760689318180084, 0.9882392883300781]",1,"[-1.9895514249801636, 1.5779130458831787]","[0.02745242789387703, 0.9725475907325745]",1
2,"[-1.9548579454421997, 2.826056718826294]","[0.008318543434143066, 0.9916815161705017]",1,"[-1.8857024908065796, 1.9852421283721924]","[0.020413290709257126, 0.979586660861969]",1,"[-1.9293957948684692, 1.5025036334991455]","[0.03131326287984848, 0.9686866998672485]",1
3,"[-1.8712629079818726, 2.656385898590088]","[0.010690528899431229, 0.9893094301223755]",1,"[-1.8154538869857788, 2.2205424308776855]","[0.01736132614314556, 0.9826387166976929]",1,"[-1.962743878364563, 1.5529305934906006]","[0.028869515284895897, 0.9711305499076843]",1
4,"[-1.6996428966522217, 2.6351511478424072]","[0.012935064733028412, 0.9870649576187134]",1,"[-2.353637933731079, 2.492295026779175]","[0.007798978127539158, 0.9922009706497192]",1,"[-2.0108678340911865, 1.5794804096221924]","[0.026848018169403076, 0.9731519818305969]",1


## Ensemble predictions based on raw predictions

In [14]:
list_sum = []

for index, row in df_xlnet.iterrows():
  pos1 = row['xlnet_raw_predictions'][0] + row['bert_raw_predictions'][0] + row['roberta_raw_predictions'][0]
  pos2 = row['xlnet_raw_predictions'][1] + row['bert_raw_predictions'][1] + row['roberta_raw_predictions'][1]
  sum = [pos1, pos2]
  list_sum.append(sum)
y_pred_raw_predicitons = np.argmax(list_sum, axis=1)

In [15]:
sub = pd.DataFrame({'id':test_data['id'], 'label':y_pred_raw_predicitons})

# write predictions to a CSV file
sub.to_csv("ensemble_raw_predictions.csv", index=False)

## Ensemble predictions based on probabilities

In [16]:
list_sum = []

for index, row in df_xlnet.iterrows():
  pos1 = row['xlnet_probabilities'][0] + row['bert_probabilities'][0] + row['roberta_probabilities'][0]
  pos2 = row['xlnet_probabilities'][1] + row['bert_probabilities'][1] + row['roberta_probabilities'][1]
  sum = [pos1/3, pos2/3]
  list_sum.append(sum)
y_pred_probabilities = np.argmax(list_sum, axis=1)

In [17]:
sub = pd.DataFrame({'id':test_data['id'], 'label':y_pred_probabilities})

# write predictions to a CSV file
sub.to_csv("ensemble_probabilities.csv", index=False)

## Ensemble predictions based on majority vote

In [18]:
labels = []

for index, row in df_xlnet.iterrows():
    if(row['xlnet_label'] + row['bert_label'] + row['roberta_label'])>= 2:
      labels.append(1)
    else:
      labels.append(0)
   

In [19]:
sub = pd.DataFrame({'id':test_data['id'], 'label':labels})

# write predictions to a CSV file
sub.to_csv("ensemble_majority_votes.csv", index=False)